In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [34]:
forums = [
    'https://forums.salary.sg/income-jobs/',
    'https://forums.salary.sg/companies/',
    'https://forums.salary.sg/cpf-central-provident-fund/',
    'https://forums.salary.sg/investments-net-worth/',
    'https://forums.salary.sg/education-personal-growth/',
    'https://forums.salary.sg/cost-living/',
    'https://forums.salary.sg/retirement/',
    'https://forums.salary.sg/bitcoin-cryptocurrency-talk/',
    'https://forums.salary.sg/skillsfuture/',
    'https://forums.salary.sg/marketplace-find-work/',
    'https://forums.salary.sg/general-discussions/',
    'https://forums.salary.sg/market-talk/'
]

In [36]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and 'thread_title' in a_.get('id', '')]
    try:
        td = soup.find_all('td', {'class': 'vbmenu_control'})
        td = [td_.text for td_ in td if 'Page 1 of ' in td_.text][0]
        max_page = int(td.split('Page 1 of ')[1])
    except:
        max_page = -1
    return a, max_page

In [32]:
get_url(urls[0])

(['https://forums.salary.sg/income-jobs/11770-compilation-monetary-authority-singapore-analysts-gross-starting-pays-graduate.html',
  'https://forums.salary.sg/income-jobs/14774-free-10-25-national-trade-union-congress-fairprice-e-voucher.html',
  'https://forums.salary.sg/income-jobs/1758-career-teacher.html',
  'https://forums.salary.sg/income-jobs/771-lawyer-salary.html',
  'https://forums.salary.sg/income-jobs/3243-aml-compliance-kyc-professionals-come.html',
  'https://forums.salary.sg/income-jobs/10265-hospital-private-public-designation-salary-range.html',
  'https://forums.salary.sg/income-jobs/793-how-life-doctor-singapore.html',
  'https://forums.salary.sg/income-jobs/2023-civil-svc-statboard-typical-timeline-making-hiring-decision.html',
  'https://forums.salary.sg/income-jobs/7152-merit-increment-civil-service.html',
  'https://forums.salary.sg/income-jobs/12886-dbs-tech-seed-programme.html',
  'https://forums.salary.sg/income-jobs/14987-what-band-5-positions-m20-21-22-23-e

In [33]:
!mkdir forums

In [40]:
max_worker = 30

forums = sorted(forums)
for i in tqdm(range(len(forums))):
    filename = os.path.join('forums', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    forum = forums[i]
    url = forum
    filtered, max_page = get_url(url)
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{url}index{a}.html', forum) for a in aranged]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_url, url[0]): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

 17%|███████▎                                    | 2/12 [00:02<00:13,  1.30s/it]
0it [00:00, ?it/s]
 25%|███████████                                 | 3/12 [00:02<00:07,  1.18it/s]
0it [00:00, ?it/s]
 83%|███████████████████████████████████▊       | 10/12 [00:50<00:13,  6.55s/it]
0it [00:00, ?it/s]
 92%|███████████████████████████████████████▍   | 11/12 [00:50<00:04,  4.60s/it]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████| 12/12 [00:50<00:00,  4.22s/it]
